In [1]:
# Import necessary packages
import numpy as np
import pylab as py
import matplotlib.pyplot as plt

# SPISEA imports
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity

# Imports for diagram colormaps 
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

# CSV parsing for reading S284 data
import csv

/home/wyz5rge/.local/lib/python3.6/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in /scratch/wyz5rge/models/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


In [2]:
# Estimation variables
star_index = 0      # Set which star in the CSV to analyze
num_top_predictions = 10     # Control how many top predictions to plot

# Isochrone instantiation

In [3]:
# define isochrone parameters
AKs = 0
dist = 4500
evo_model = evolution.Baraffe15()
atm_func = atmospheres.get_merged_atmosphere
red_law = reddening.RedLawHosek18b()
filt_list = ['jwst,F162M', 'jwst,F182M', 'jwst,F200W', 'jwst,F356W', 'jwst,F405N']
filters = ['m_jwst_F162M', 'm_jwst_F182M', 'm_jwst_F200W', 'm_jwst_F356W', 'm_jwst_F405N']
iso_dir = 'isochrones/'
metallicity = 0

In [4]:
# define log age array for isochrone grid
level_ages = np.linspace(1, 10, 19) * 1e6
log_age_arr = np.log10(level_ages)

In [5]:
instances = np.array([
    synthetic.IsochronePhot(log_age, AKs, dist, metallicity=metallicity,
                            evo_model=evo_model, atm_func=atm_func,
                            red_law=red_law, filters=filt_list,
                            iso_dir=iso_dir)
    for log_age in log_age_arr
])

Isochrone generation took 1.131397 s.
Making photometry for isochrone: log(t) = 6.00  AKs = 0.00  dist = 4500
     Starting at:  2024-10-28 00:22:53.115807   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F162M = 21.99
Starting filter: jwst,F182M   Elapsed time: 0.63 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F182M = 22.18
Starting filter: jwst,F200W   Elapsed time: 1.26 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F200W = 22.04
Starting filter: jwst,F356W   Elapsed time: 1.88 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F356W = 21.17
Starting filter: jwst,F405N   Elapsed time: 2.54 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F405N = 20.84
      Time taken: 3.22 seconds
Isochrone generation took 1.022754 s.
Making photometry for isochrone: log(t) = 6.18  AKs =

      Time taken: 3.25 seconds
Changing to logg=5.00 for T=  1893 logg=3.97
Isochrone generation took 1.032349 s.
Making photometry for isochrone: log(t) = 6.74  AKs = 0.00  dist = 4500
     Starting at:  2024-10-28 00:23:32.093903   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1893 K  m_jwst_F162M = 23.66
Starting filter: jwst,F182M   Elapsed time: 0.63 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1893 K  m_jwst_F182M = 24.05
Starting filter: jwst,F200W   Elapsed time: 1.25 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1893 K  m_jwst_F200W = 23.79
Starting filter: jwst,F356W   Elapsed time: 1.87 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1893 K  m_jwst_F356W = 22.54
Starting filter: jwst,F405N   Elapsed time: 2.53 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1893 K  m_jwst_F405N = 22.18
      Time taken: 3.20 seconds
Changing to log

Starting filter: jwst,F356W   Elapsed time: 1.87 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1602 K  m_jwst_F356W = 23.04
Starting filter: jwst,F405N   Elapsed time: 2.53 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1602 K  m_jwst_F405N = 22.62
      Time taken: 3.21 seconds
Changing to logg=5.00 for T=  1571 logg=4.05
Isochrone generation took 1.090661 s.
Making photometry for isochrone: log(t) = 7.00  AKs = 0.00  dist = 4500
     Starting at:  2024-10-28 00:24:10.532968   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1571 K  m_jwst_F162M = 25.19
Starting filter: jwst,F182M   Elapsed time: 0.63 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1571 K  m_jwst_F182M = 25.03
Starting filter: jwst,F200W   Elapsed time: 1.26 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1571 K  m_jwst_F200W = 24.71
Starting filter: jwst,F356W   Elapsed time: 1.

# Function for calculating chi-square value based on photometric magnitudes

In [6]:
# Chi-square minimization function
def chi_square_reverse_model(iso_grid, sample_mags):
    results = []
    for i, iso in enumerate(iso_grid):
        for star in iso.points:
            chi_square = sum(((sample_mags[k] - star[filters[k]]) ** 2) / star[filters[k]] for k in range(len(sample_mags)))
            results.append([chi_square, star['mass'], 10 ** log_age_arr[i]])
    
    return sorted(results, key=lambda x: x[0])

# Define S284 sample magnitudes and run function

In [7]:
# Load sample magnitudes, skipping the header row
sample_mags = []
with open('../s284-no-errors.csv', mode='r') as file:
    csvFile = csv.reader(file)
    next(csvFile)  # Skip header row
    for lines in csvFile:
        sample_mags.append([float(x) for x in lines[1:]])  # Convert each entry to float

FileNotFoundError: [Errno 2] No such file or directory: '../s284-no-errors.csv'

In [ ]:
# Run chi-square minimization and store top results
sorted_results = chi_square_reverse_model(instances, sample_mags[0])
top_results = sorted_results[:num_top_predictions]  # For example, we take the top 10 best matches

In [ ]:
# Generate plot
fig, ax = plt.subplots(figsize=(10, 10))
cmap = plt.get_cmap('coolwarm')

# Plot all isochrones with colors based on age
for i, instance in enumerate(instances):
    color = cmap(i / (len(instances) - 1))  # Assign color based on index and colormap
    ax.plot(instance.points[filters[0]], instance.points[filters[1]], color=color)

ax.set_xlabel('F162M')
ax.set_ylabel('F182M')
ax.invert_yaxis()

# Create colorbar legend
norm = Normalize(vmin=min(level_ages), vmax=max(level_ages))
sm = ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])
cbar = plt.colorbar(sm)
cbar.set_label('Age (millions of years)')
cbar.set_ticks(level_ages)
cbar.set_ticklabels([f'{age/1e6:.1f}' for age in level_ages])

# Plot each estimated star's position in red on the diagram
for j, (chi_square, mass, age) in top_results:
    # Find the closest age in level_ages to the estimated age
    closest_age_idx = np.argmin(abs(level_ages - age))
    iso_instance = instances[closest_age_idx]  # Access the isochrone instance

    # Find the star in the isochrone closest to the estimated mass
    mass_idx = np.argmin(abs(iso_instance.points['mass'] - mass))
    
    # Plot the star's position on the color-magnitude diagram
    ax.plot(iso_instance.points[mass_idx][filters[0]], iso_instance.points[mass_idx][filters[1]], 'o', color='red')

    # Plot each estimated star point with color gradient and labels
    ax.plot(
        iso_instance.points[mass_idx][filters[0]],
        iso_instance.points[mass_idx][filters[1]],
        'o', color=cmap(j / num_top_predictions),
        label=f"Est. Mass: {mass:.2f}, Age: {age/1e6:.1f} Myr"
    )

# Plot reference star
ax.plot(sample_mags[star_index][0], sample_mags[star_index][1], '*', color='purple', 
        markersize=10, label="Reference Star")
    
ax.legend()
    
plt.show()